In [108]:
import polars as pl
import matplotlib.pyplot as plt

In [111]:
df = pl.read_csv('./log-3.csv')

In [112]:
df.head()

chosen_dag,num_clients,num_samples,comparison_category,same_p_val,missed_tests,total_features,features_per_client,tikhonov_lambda,correctness_alpha_0.01,correctness_alpha_0.05,correctness_alpha_0.1
str,i64,i64,str,f64,f64,i64,i64,f64,f64,f64,f64
"""chain4""",1,100,"""ground_truth""",0.166667,0.0,4,4,0.0000001,0.895833,0.833333,0.854167
"""chain4""",3,100,"""ground_truth""",0.166667,0.0,4,4,0.0000001,0.895833,0.854167,0.854167
"""chain4""",5,100,"""ground_truth""",0.166667,0.0,4,4,0.0000001,0.895833,0.833333,0.854167
"""chain4""",10,100,"""ground_truth""",0.125,0.0,4,4,0.0000001,0.8125,0.8125,0.770833
"""chain4""",1,100,"""real""",0.166667,0.0,4,4,0.0000001,0.583333,0.666667,0.666667


In [121]:
def filter_to_complete_featuresets_per_client(df):
    return df.filter(pl.col('total_features') == pl.col('features_per_client'))

def filter_to_incomplete_featuresets_per_client(df):
    return df.filter(pl.col('total_features') != pl.col('features_per_client'))

def filter_to_citest_comparison(df):
    return df.filter(pl.col('comparison_category') == 'ground_truth')

def filter_to_real_comparison(df):
    return df.filter(pl.col('comparison_category') == 'real')

In [124]:
_df = df
_df = filter_to_complete_featuresets_per_client(_df)
_df = filter_to_real_comparison(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.01', by=['num_clients', 'num_samples'], c='num_clients', cmap='Pastel2')

:BoxWhisker   [num_clients,num_samples]   (correctness_alpha_0.01)

In [125]:
_df = df
_df = filter_to_complete_featuresets_per_client(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.01', 
             by=['comparison_category', 'num_clients', 'num_samples'],
             c='num_clients',
             cmap='Pastel2')

:BoxWhisker   [comparison_category,num_clients,num_samples]   (correctness_alpha_0.01)

# Tikhonov Regularization Analysis

In [138]:
import polars as pl

In [139]:
file_id = 1
df = pl.read_csv(f'./log-{file_id}.csv')
df_continuos = pl.read_csv(f'./log-8{file_id}.csv')

In [141]:
_df = df
_df = filter_to_complete_featuresets_per_client(_df)
#_df = filter_to_real_comparison(_df)
_df = filter_to_citest_comparison(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.01',
             by=['num_clients', 'num_samples', 'tikhonov_lambda'],
             c='num_clients',
             cmap='Pastel2',
             ylim=(-0.1,1.1)
             )

:BoxWhisker   [num_clients,num_samples,tikhonov_lambda]   (correctness_alpha_0.01)

In [142]:
_df = df_continuos
_df = filter_to_complete_featuresets_per_client(_df)
#_df = filter_to_real_comparison(_df)
_df = filter_to_citest_comparison(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.01',
             by=['num_clients', 'num_samples', 'tikhonov_lambda'],
             c='num_clients',
             cmap='Pastel2',
             ylim=(-0.1,1.1)
             )

:BoxWhisker   [num_clients,num_samples,tikhonov_lambda]   (correctness_alpha_0.01)